In [2]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import dateutil.parser as dp
from datetime import datetime
from datetime import timedelta  
import time
from datetime import timezone
session = Session()
import os

import numpy as np

#converts pyton datatime to unix milisecond timestamp utc timezone
def to_utcunix(x):
    return int(x.replace(tzinfo=timezone.utc).timestamp()) * 1000

#Generator for binance price data
def get_data(start, batch_size, end = datetime.now(), symbol = "BTCUSDT", interval = "1m"):
    batch_start = start
    
    while True:
        batch_end = batch_start + timedelta(minutes = batch_size)
        
        if(batch_end > end):
            batch_end = end - timedelta(minutes = 1)
        
            if(batch_start >= end- timedelta(minutes = 1)):
                break
        
        base = "https://api.binance.com/api/v3"

        print(batch_start, batch_end)
        
        url = base + "/klines?symbol={}&interval={}&startTime={}&endTime={}&limit={}".format(symbol, interval,
                                                                                     to_utcunix(batch_start)
                                                                                    , to_utcunix(batch_end), batch_size)
        
        
        batch_start = batch_end
        #print(url)
        #sleep to stay under binance maximum call rate
        time.sleep(0.15)
        
        while True:
            try:
                data = session.get(url).json()
                #print(data)
                yield pd.DataFrame.from_dict(data)[[0,1,2,3,4,5]]
            except:
                print("Retrieval error")
                yield pd.DataFrame(np.zeros([batch_size, 6]) * np.nan)
                
                #time.sleep(10)
                #continue
            break

#save binance price data as csv
def gen_csv(start, filename, end = datetime.now(), symbol = "BTCUSDT", interval = "1m"):
    batch_size = 1000
    d = get_data(start, batch_size, end = end, symbol = symbol, interval = interval)
        
    try:
        os.mkdir("data_" + interval)
    except:
        pass
    
    fn = "data_" + interval + "/" + filename + ".csv"
    pd.DataFrame([["Open Time", "Open", "High",  "Low",  "Close",  "Volume"]]).to_csv(fn, mode = 'a', header = None)
    for i in d:
        i.to_csv(fn, mode = 'a', header = None)#, header = ["Open Time", "Open", "High",  "Low",  "Close",  "Volume"])

#simplified gen_csv function
def gen_csv_year(year, symbol, interval = "1m"):
    gen_csv(datetime(year,1,1) , "{}_{}".format(year, symbol) ,datetime(year + 1,1,1), symbol, interval )
    

#for i in range(2016, 2021):
#    gen_csv_year(i, "ETHUSDT")    


In [30]:

for s in ['ETHUSDT', 'BTCUSDT']:
    gen_csv_year(2018, s, interval = "5m") 

for y in range(2018, 2021):
    for s in ['XRPUSDT', 'BCHUSDT', "LTCUSDT"]:
        gen_csv_year(y, s, interval = "5m") 

2018-01-01 00:00:00 2018-01-01 16:40:00
2018-01-01 16:40:00 2018-01-02 09:20:00
2018-01-02 09:20:00 2018-01-03 02:00:00
2018-01-03 02:00:00 2018-01-03 18:40:00
2018-01-03 18:40:00 2018-01-04 11:20:00
2018-01-04 11:20:00 2018-01-05 04:00:00
2018-01-05 04:00:00 2018-01-05 20:40:00
2018-01-05 20:40:00 2018-01-06 13:20:00
2018-01-06 13:20:00 2018-01-07 06:00:00
2018-01-07 06:00:00 2018-01-07 22:40:00
2018-01-07 22:40:00 2018-01-08 15:20:00
2018-01-08 15:20:00 2018-01-09 08:00:00
2018-01-09 08:00:00 2018-01-10 00:40:00
2018-01-10 00:40:00 2018-01-10 17:20:00
2018-01-10 17:20:00 2018-01-11 10:00:00
2018-01-11 10:00:00 2018-01-12 02:40:00
2018-01-12 02:40:00 2018-01-12 19:20:00
2018-01-12 19:20:00 2018-01-13 12:00:00
2018-01-13 12:00:00 2018-01-14 04:40:00
2018-01-14 04:40:00 2018-01-14 21:20:00
2018-01-14 21:20:00 2018-01-15 14:00:00
2018-01-15 14:00:00 2018-01-16 06:40:00
2018-01-16 06:40:00 2018-01-16 23:20:00
2018-01-16 23:20:00 2018-01-17 16:00:00
2018-01-17 16:00:00 2018-01-18 08:40:00


2018-05-23 08:40:00 2018-05-24 01:20:00
2018-05-24 01:20:00 2018-05-24 18:00:00
2018-05-24 18:00:00 2018-05-25 10:40:00
2018-05-25 10:40:00 2018-05-26 03:20:00
2018-05-26 03:20:00 2018-05-26 20:00:00
2018-05-26 20:00:00 2018-05-27 12:40:00
2018-05-27 12:40:00 2018-05-28 05:20:00
2018-05-28 05:20:00 2018-05-28 22:00:00
2018-05-28 22:00:00 2018-05-29 14:40:00
2018-05-29 14:40:00 2018-05-30 07:20:00
2018-05-30 07:20:00 2018-05-31 00:00:00
2018-05-31 00:00:00 2018-05-31 16:40:00
2018-05-31 16:40:00 2018-06-01 09:20:00
2018-06-01 09:20:00 2018-06-02 02:00:00
2018-06-02 02:00:00 2018-06-02 18:40:00
2018-06-02 18:40:00 2018-06-03 11:20:00
2018-06-03 11:20:00 2018-06-04 04:00:00
2018-06-04 04:00:00 2018-06-04 20:40:00
2018-06-04 20:40:00 2018-06-05 13:20:00
2018-06-05 13:20:00 2018-06-06 06:00:00
2018-06-06 06:00:00 2018-06-06 22:40:00
2018-06-06 22:40:00 2018-06-07 15:20:00
2018-06-07 15:20:00 2018-06-08 08:00:00
2018-06-08 08:00:00 2018-06-09 00:40:00
2018-06-09 00:40:00 2018-06-09 17:20:00


2018-10-12 17:20:00 2018-10-13 10:00:00
2018-10-13 10:00:00 2018-10-14 02:40:00
2018-10-14 02:40:00 2018-10-14 19:20:00
2018-10-14 19:20:00 2018-10-15 12:00:00
2018-10-15 12:00:00 2018-10-16 04:40:00
2018-10-16 04:40:00 2018-10-16 21:20:00
2018-10-16 21:20:00 2018-10-17 14:00:00
2018-10-17 14:00:00 2018-10-18 06:40:00
2018-10-18 06:40:00 2018-10-18 23:20:00
2018-10-18 23:20:00 2018-10-19 16:00:00
2018-10-19 16:00:00 2018-10-20 08:40:00
2018-10-20 08:40:00 2018-10-21 01:20:00
2018-10-21 01:20:00 2018-10-21 18:00:00
2018-10-21 18:00:00 2018-10-22 10:40:00
2018-10-22 10:40:00 2018-10-23 03:20:00
2018-10-23 03:20:00 2018-10-23 20:00:00
2018-10-23 20:00:00 2018-10-24 12:40:00
2018-10-24 12:40:00 2018-10-25 05:20:00
2018-10-25 05:20:00 2018-10-25 22:00:00
2018-10-25 22:00:00 2018-10-26 14:40:00
2018-10-26 14:40:00 2018-10-27 07:20:00
2018-10-27 07:20:00 2018-10-28 00:00:00
2018-10-28 00:00:00 2018-10-28 16:40:00
2018-10-28 16:40:00 2018-10-29 09:20:00
2018-10-29 09:20:00 2018-10-30 02:00:00


2018-03-03 19:20:00 2018-03-04 12:00:00
2018-03-04 12:00:00 2018-03-05 04:40:00
2018-03-05 04:40:00 2018-03-05 21:20:00
2018-03-05 21:20:00 2018-03-06 14:00:00
2018-03-06 14:00:00 2018-03-07 06:40:00
2018-03-07 06:40:00 2018-03-07 23:20:00
2018-03-07 23:20:00 2018-03-08 16:00:00
2018-03-08 16:00:00 2018-03-09 08:40:00
2018-03-09 08:40:00 2018-03-10 01:20:00
2018-03-10 01:20:00 2018-03-10 18:00:00
2018-03-10 18:00:00 2018-03-11 10:40:00
2018-03-11 10:40:00 2018-03-12 03:20:00
2018-03-12 03:20:00 2018-03-12 20:00:00
2018-03-12 20:00:00 2018-03-13 12:40:00
2018-03-13 12:40:00 2018-03-14 05:20:00
2018-03-14 05:20:00 2018-03-14 22:00:00
2018-03-14 22:00:00 2018-03-15 14:40:00
2018-03-15 14:40:00 2018-03-16 07:20:00
2018-03-16 07:20:00 2018-03-17 00:00:00
2018-03-17 00:00:00 2018-03-17 16:40:00
2018-03-17 16:40:00 2018-03-18 09:20:00
2018-03-18 09:20:00 2018-03-19 02:00:00
2018-03-19 02:00:00 2018-03-19 18:40:00
2018-03-19 18:40:00 2018-03-20 11:20:00
2018-03-20 11:20:00 2018-03-21 04:00:00


2018-07-24 04:00:00 2018-07-24 20:40:00
2018-07-24 20:40:00 2018-07-25 13:20:00
2018-07-25 13:20:00 2018-07-26 06:00:00
2018-07-26 06:00:00 2018-07-26 22:40:00
2018-07-26 22:40:00 2018-07-27 15:20:00
2018-07-27 15:20:00 2018-07-28 08:00:00
2018-07-28 08:00:00 2018-07-29 00:40:00
2018-07-29 00:40:00 2018-07-29 17:20:00
2018-07-29 17:20:00 2018-07-30 10:00:00
2018-07-30 10:00:00 2018-07-31 02:40:00
2018-07-31 02:40:00 2018-07-31 19:20:00
2018-07-31 19:20:00 2018-08-01 12:00:00
2018-08-01 12:00:00 2018-08-02 04:40:00
2018-08-02 04:40:00 2018-08-02 21:20:00
2018-08-02 21:20:00 2018-08-03 14:00:00
2018-08-03 14:00:00 2018-08-04 06:40:00
2018-08-04 06:40:00 2018-08-04 23:20:00
2018-08-04 23:20:00 2018-08-05 16:00:00
2018-08-05 16:00:00 2018-08-06 08:40:00
2018-08-06 08:40:00 2018-08-07 01:20:00
2018-08-07 01:20:00 2018-08-07 18:00:00
2018-08-07 18:00:00 2018-08-08 10:40:00
2018-08-08 10:40:00 2018-08-09 03:20:00
2018-08-09 03:20:00 2018-08-09 20:00:00
2018-08-09 20:00:00 2018-08-10 12:40:00


...

2020-06-28 20:40:00 2020-06-29 13:20:00
2020-06-29 13:20:00 2020-06-30 06:00:00
2020-06-30 06:00:00 2020-06-30 22:40:00
2020-06-30 22:40:00 2020-07-01 15:20:00
2020-07-01 15:20:00 2020-07-02 08:00:00
2020-07-02 08:00:00 2020-07-03 00:40:00
2020-07-03 00:40:00 2020-07-03 17:20:00
2020-07-03 17:20:00 2020-07-04 10:00:00
2020-07-04 10:00:00 2020-07-05 02:40:00
2020-07-05 02:40:00 2020-07-05 19:20:00
2020-07-05 19:20:00 2020-07-06 12:00:00
2020-07-06 12:00:00 2020-07-07 04:40:00
2020-07-07 04:40:00 2020-07-07 21:20:00
2020-07-07 21:20:00 2020-07-08 14:00:00
2020-07-08 14:00:00 2020-07-09 06:40:00
2020-07-09 06:40:00 2020-07-09 23:20:00
2020-07-09 23:20:00 2020-07-10 16:00:00
2020-07-10 16:00:00 2020-07-11 08:40:00
2020-07-11 08:40:00 2020-07-12 01:20:00
2020-07-12 01:20:00 2020-07-12 18:00:00
2020-07-12 18:00:00 2020-07-13 10:40:00
2020-07-13 10:40:00 2020-07-14 03:20:00
2020-07-14 03:20:00 2020-07-14 20:00:00
2020-07-14 20:00:00 2020-07-15 12:40:00
2020-07-15 12:40:00 2020-07-16 05:20:00


Retrieval error
2020-10-25 14:40:00 2020-10-26 07:20:00
Retrieval error
2020-10-26 07:20:00 2020-10-27 00:00:00
Retrieval error
2020-10-27 00:00:00 2020-10-27 16:40:00
Retrieval error
2020-10-27 16:40:00 2020-10-28 09:20:00
Retrieval error
2020-10-28 09:20:00 2020-10-29 02:00:00
Retrieval error
2020-10-29 02:00:00 2020-10-29 18:40:00
Retrieval error
2020-10-29 18:40:00 2020-10-30 11:20:00
Retrieval error
2020-10-30 11:20:00 2020-10-31 04:00:00
Retrieval error
2020-10-31 04:00:00 2020-10-31 20:40:00
Retrieval error
2020-10-31 20:40:00 2020-11-01 13:20:00
Retrieval error
2020-11-01 13:20:00 2020-11-02 06:00:00
Retrieval error
2020-11-02 06:00:00 2020-11-02 22:40:00
Retrieval error
2020-11-02 22:40:00 2020-11-03 15:20:00
Retrieval error
2020-11-03 15:20:00 2020-11-04 08:00:00
Retrieval error
2020-11-04 08:00:00 2020-11-05 00:40:00
Retrieval error
2020-11-05 00:40:00 2020-11-05 17:20:00
Retrieval error
2020-11-05 17:20:00 2020-11-06 10:00:00
Retrieval error
2020-11-06 10:00:00 2020-11-07 0

2020-02-17 05:20:00 2020-02-17 22:00:00
2020-02-17 22:00:00 2020-02-18 14:40:00
2020-02-18 14:40:00 2020-02-19 07:20:00
2020-02-19 07:20:00 2020-02-20 00:00:00
2020-02-20 00:00:00 2020-02-20 16:40:00
2020-02-20 16:40:00 2020-02-21 09:20:00
2020-02-21 09:20:00 2020-02-22 02:00:00
2020-02-22 02:00:00 2020-02-22 18:40:00
2020-02-22 18:40:00 2020-02-23 11:20:00
2020-02-23 11:20:00 2020-02-24 04:00:00
2020-02-24 04:00:00 2020-02-24 20:40:00
2020-02-24 20:40:00 2020-02-25 13:20:00
2020-02-25 13:20:00 2020-02-26 06:00:00
2020-02-26 06:00:00 2020-02-26 22:40:00
2020-02-26 22:40:00 2020-02-27 15:20:00
2020-02-27 15:20:00 2020-02-28 08:00:00
2020-02-28 08:00:00 2020-02-29 00:40:00
2020-02-29 00:40:00 2020-02-29 17:20:00
2020-02-29 17:20:00 2020-03-01 10:00:00
2020-03-01 10:00:00 2020-03-02 02:40:00
2020-03-02 02:40:00 2020-03-02 19:20:00
2020-03-02 19:20:00 2020-03-03 12:00:00
2020-03-03 12:00:00 2020-03-04 04:40:00
2020-03-04 04:40:00 2020-03-04 21:20:00
2020-03-04 21:20:00 2020-03-05 14:00:00


2020-07-08 14:00:00 2020-07-09 06:40:00
2020-07-09 06:40:00 2020-07-09 23:20:00
2020-07-09 23:20:00 2020-07-10 16:00:00
2020-07-10 16:00:00 2020-07-11 08:40:00
2020-07-11 08:40:00 2020-07-12 01:20:00
2020-07-12 01:20:00 2020-07-12 18:00:00
2020-07-12 18:00:00 2020-07-13 10:40:00
2020-07-13 10:40:00 2020-07-14 03:20:00
2020-07-14 03:20:00 2020-07-14 20:00:00
2020-07-14 20:00:00 2020-07-15 12:40:00
2020-07-15 12:40:00 2020-07-16 05:20:00
2020-07-16 05:20:00 2020-07-16 22:00:00
2020-07-16 22:00:00 2020-07-17 14:40:00
2020-07-17 14:40:00 2020-07-18 07:20:00
2020-07-18 07:20:00 2020-07-19 00:00:00
2020-07-19 00:00:00 2020-07-19 16:40:00
2020-07-19 16:40:00 2020-07-20 09:20:00
2020-07-20 09:20:00 2020-07-21 02:00:00
2020-07-21 02:00:00 2020-07-21 18:40:00
2020-07-21 18:40:00 2020-07-22 11:20:00
2020-07-22 11:20:00 2020-07-23 04:00:00
2020-07-23 04:00:00 2020-07-23 20:40:00
2020-07-23 20:40:00 2020-07-24 13:20:00
2020-07-24 13:20:00 2020-07-25 06:00:00
2020-07-25 06:00:00 2020-07-25 22:40:00


Retrieval error
2020-11-01 13:20:00 2020-11-02 06:00:00
Retrieval error
2020-11-02 06:00:00 2020-11-02 22:40:00
Retrieval error
2020-11-02 22:40:00 2020-11-03 15:20:00
Retrieval error
2020-11-03 15:20:00 2020-11-04 08:00:00
Retrieval error
2020-11-04 08:00:00 2020-11-05 00:40:00
Retrieval error
2020-11-05 00:40:00 2020-11-05 17:20:00
Retrieval error
2020-11-05 17:20:00 2020-11-06 10:00:00
Retrieval error
2020-11-06 10:00:00 2020-11-07 02:40:00
Retrieval error
2020-11-07 02:40:00 2020-11-07 19:20:00
Retrieval error
2020-11-07 19:20:00 2020-11-08 12:00:00
Retrieval error
2020-11-08 12:00:00 2020-11-09 04:40:00
Retrieval error
2020-11-09 04:40:00 2020-11-09 21:20:00
Retrieval error
2020-11-09 21:20:00 2020-11-10 14:00:00
Retrieval error
2020-11-10 14:00:00 2020-11-11 06:40:00
Retrieval error
2020-11-11 06:40:00 2020-11-11 23:20:00
Retrieval error
2020-11-11 23:20:00 2020-11-12 16:00:00
Retrieval error
2020-11-12 16:00:00 2020-11-13 08:40:00
Retrieval error
2020-11-13 08:40:00 2020-11-14 0

In [28]:
np.zeros([5, 6]) * np.nan

array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]])

In [4]:
#for s in ['ETHUSDT', 'BTCUSDT']:
gen_csv_year(2017, 'BTCUSDT', interval = "5m") 

2017-01-01 00:00:00 2017-01-01 16:40:00
Retrieval error
2017-01-01 16:40:00 2017-01-02 09:20:00
Retrieval error
2017-01-02 09:20:00 2017-01-03 02:00:00
Retrieval error
2017-01-03 02:00:00 2017-01-03 18:40:00
Retrieval error
2017-01-03 18:40:00 2017-01-04 11:20:00
Retrieval error
2017-01-04 11:20:00 2017-01-05 04:00:00
Retrieval error
2017-01-05 04:00:00 2017-01-05 20:40:00
Retrieval error
2017-01-05 20:40:00 2017-01-06 13:20:00
Retrieval error
2017-01-06 13:20:00 2017-01-07 06:00:00
Retrieval error
2017-01-07 06:00:00 2017-01-07 22:40:00
Retrieval error
2017-01-07 22:40:00 2017-01-08 15:20:00


KeyboardInterrupt: 